# Homework 6: 1, 2, 9 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ISLP import load_data
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.cross_decomposition import PLSRegression

## Load Data 

In [4]:
df= load_data("college")

In [6]:
df.head()

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15


In [ ]:
# Split the dataset into training and testing sets
train, test  = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Separate features and target variable
X_train = train.drop(columns=['Apps'])
y_train = train['Apps']

X_test = test.drop(columns=['Apps'])
y_test = test['Apps']

## Preprocessed Stage 

In [18]:
# Optional: separate numeric and categorical
transformer = make_column_transformer(
    (OneHotEncoder(drop="first"), ['Private']),
    remainder='passthrough'
)
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder', OneHotEncoder(drop='first'),
                                 ['Private'])])

## Linear Regression 

In [20]:
model  = make_pipeline(transformer, LinearRegression())
model.fit(X_train, y_train)

/Users/abdoulabdillahi/miniconda3/envs/env-statistic/lib/python3.9/site-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['Private'])])),
                ('linearregression', LinearRegression())])

In [24]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2_score = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print("Test MAE:", mae)
print("Test R2 Score:", r2_score)
print("Test MAPE:", mape)
print("Test RMSE:", rmse)


Test MAE: 744.8589553385909
Test R2 Score: 0.8877583168400972
Test MAPE: 0.3776211861052174
Test RMSE: 1221.655998650623


## Ridge Regression 

In [29]:

# Step 1: Create pipeline with preprocessing and RidgeCV
alphas = np.logspace(-3, 3, 100)  # Try lambda values from 0.001 to 1000

ridge_model = make_pipeline(
    transformer,
    RidgeCV(alphas=alphas, store_cv_values=True)
)

# Step 2: Fit model to training data
ridge_model.fit(X_train, y_train)

# Step 3: Predict on test set
y_pred_ridge = ridge_model.predict(X_test)

# Step 4: Calculate test RMSE
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
r2_score_ridge = r2_score(y_test, y_pred_ridge)
mape_ridge = mean_absolute_percentage_error(y_test, y_pred_ridge)

print("Test MAE (Ridge):", mae_ridge)
print("Test R2 Score (Ridge):", r2_score_ridge)
print("Test MAPE (Ridge):", mape_ridge)
print("Test RMSE (Ridge):", rmse_ridge)

# Optional: Best lambda (alpha) value selected by cross-validation
best_alpha = ridge_model.named_steps['ridgecv'].alpha_
print("Best alpha (λ) chosen by CV:", best_alpha)

/Users/abdoulabdillahi/miniconda3/envs/env-statistic/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


Test MAE (Ridge): 737.5014390129802
Test R2 Score (Ridge): 0.8888019875006756
Test MAPE (Ridge): 0.36870960335660624
Test RMSE (Ridge): 1215.9629965723898
Best alpha (λ) chosen by CV: 10.0


## Lasso 

In [32]:
# Step 1: Set range of lambda (alpha) values to test
alphas = np.logspace(-3, 1, 100)  # λ values from 0.001 to 10

# Step 2: Create pipeline with transformer and LassoCV
lasso_model = make_pipeline(
    transformer,
    LassoCV(alphas=alphas, cv=5, max_iter=10000)
)

# Step 3: Fit the model
lasso_model.fit(X_train, y_train)

# Step 4: Predict on test set
y_pred_lasso = lasso_model.predict(X_test)

# Step 5: Compute test RMSE
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
print("Test RMSE (Lasso):", rmse_lasso)
print("Test MAE (Lasso):", mean_absolute_error(y_test, y_pred_lasso))
print("Test R2 Score (Lasso):", r2_score(y_test, y_pred_lasso))
print("Test MAPE (Lasso):", mean_absolute_percentage_error(y_test, y_pred_lasso))

# Step 6: Get best alpha (λ)
best_alpha_lasso = lasso_model.named_steps['lassocv'].alpha_
print("Best alpha (λ) chosen by CV:", best_alpha_lasso)

# Step 7: Count number of non-zero coefficients
non_zero_coefs = np.sum(lasso_model.named_steps['lassocv'].coef_ != 0)
print("Number of non-zero coefficients:", non_zero_coefs)

Test RMSE (Lasso): 1216.8313944387187
Test MAE (Lasso): 738.1064788503411
Test R2 Score (Lasso): 0.8886431033925093
Test MAPE (Lasso): 0.36943813175416385
Best alpha (λ) chosen by CV: 7.56463327554629
Number of non-zero coefficients: 17


## Principal Components Regression  Model 

In [37]:

# Step 1: Standardize + encode using your transformer
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

# Step 2: Loop over possible M values (1 to max components)
n_components = X_train_transformed.shape[1]
rmse_cv = []

for m in range(1, n_components + 1):
    # Create pipeline: PCA -> LinearRegression
    pcr_pipeline = make_pipeline(
        PCA(n_components=m),
        LinearRegression()
    )
    
    # Negative MSE from cross_val_score (5-fold CV)
    neg_mse = cross_val_score(
        pcr_pipeline,
        X_train_transformed,
        y_train,
        scoring='neg_mean_squared_error',
        cv=5
    )
    
    # Compute mean RMSE and store
    rmse = np.sqrt(-neg_mse.mean())
    rmse_cv.append(rmse)

# Step 3: Find best M (minimum RMSE)
best_m = np.argmin(rmse_cv) + 1  # +1 because range starts at 1
print("Best number of components (M):", best_m)

# Step 4: Fit PCR model using best M
final_pcr = make_pipeline(
    PCA(n_components=best_m),
    LinearRegression()
)
final_pcr.fit(X_train_transformed, y_train)

# Step 5: Predict on test set and compute test RMSE
y_pred_pcr = final_pcr.predict(X_test_transformed)
rmse_test_pcr = np.sqrt(mean_squared_error(y_test, y_pred_pcr))
print("Test RMSE (PCR):", rmse_test_pcr)
print("Test MAE (PCR):", mean_absolute_error(y_test, y_pred_pcr))
print("Test R2 Score (PCR):", r2_score(y_test, y_pred_pcr))
print("Test MAPE (PCR):", mean_absolute_percentage_error(y_test, y_pred_pcr))   

Best number of components (M): 17
Test RMSE (PCR): 1221.65599865061
Test MAE (PCR): 744.8589553385866
Test R2 Score (PCR): 0.8877583168400996
Test MAPE (PCR): 0.37762118610521533


## PLS With Cross Validation 

In [42]:
 #Step 1: Transform features using the same transformer
X_train_transformed = transformer.fit_transform(X_train)
X_test_transformed = transformer.transform(X_test)

# Step 2: Loop over possible M values (1 to number of features)
n_components = X_train_transformed.shape[1]
rmse_cv = []

for m in range(1, n_components + 1):
    pls = PLSRegression(n_components=m)
    scores = cross_val_score(
        pls,
        X_train_transformed,
        y_train,
        scoring='neg_mean_squared_error',
        cv=5
    )
    rmse = np.sqrt(-scores.mean())
    rmse_cv.append(rmse)

# Step 3: Find best number of components
best_m = np.argmin(rmse_cv) + 1
print("Best number of PLS components (M):", best_m)

# Step 4: Fit final PLS model using best M
pls_final = PLSRegression(n_components=best_m)
pls_final.fit(X_train_transformed, y_train)

# Step 5: Predict on test set and compute RMSE
y_pred_pls = pls_final.predict(X_test_transformed)
rmse_test_pls = np.sqrt(mean_squared_error(y_test, y_pred_pls))
print("Test RMSE (PLS):", rmse_test_pls)
print("Test MAE (PLS):", mean_absolute_error(y_test, y_pred_pls))
print("Test R2 Score (PLS):", r2_score(y_test, y_pred_pls))   
print("Test MAPE (PLS):", mean_absolute_percentage_error(y_test, y_pred_pls))      

Best number of PLS components (M): 17
Test RMSE (PLS): 1221.655998650612
Test MAE (PLS): 744.8589553385873
Test R2 Score (PLS): 0.8877583168400992
Test MAPE (PLS): 0.3776211861052158
